In [6]:
# caption prediction
import open_clip
import torch
from PIL import Image
#('coca_ViT-B-32', 'laion2b_s13b_b90k')
# model, _, transform = open_clip.create_model_and_transforms(
#     model_name="coca_ViT-B-32",  #coca_ViT-L-14, 
#     pretrained="laion2b_s13b_b90k", #mscoco_finetuned_laion2B-s13B-b90k
# )
# /mnt/eds_share/Users/yilu.zhou/Development/log/open_clip/2023_07_28-13_55_44-model_coca_ViT-L-14-lr_1e-05-b_32-j_4-p_amp/checkpoints/epoch_1.pt
model, _, transform = open_clip.create_model_and_transforms(
    model_name="coca_ViT-L-14",
    pretrained="/mnt/eds_share/Users/yilu.zhou/Development/log/open_clip_GlobusSrgMapData_crop_square/2023_08_08-22_02_21-model_coca_ViT-L-14-lr_1e-06-b_32-j_4-p_amp/checkpoints/epoch_78.pt"
)


In [7]:
import torch
print(torch.__version__)

2.1.0.dev20230816+cu118


In [8]:
import cv2
import torch
from albumentations import Resize, Compose
from albumentations.pytorch.transforms import  ToTensor
from albumentations.augmentations.transforms import Normalize

tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
sentences = ["thoracic", "lumbar", "thoracic lumbar"]
dummy_texts = tokenizer(sentences) 

image_path = '/mnt/eds_share/share/Spine2D/GlobusSrgMapData_crop_square/test/images/anon_d4730414.dcm_lateral_full_lumbar_thoracic.jpg'
im = Image.open(image_path).convert("RGB")
im = im.resize((224, 224), Image.Resampling.LANCZOS)
dummy_image = transform(im).unsqueeze(0)



ONNX_FILE_PATH = 'openclip.onnx'
torch.onnx.export(model, (dummy_image, dummy_texts), ONNX_FILE_PATH, input_names=["IMAGE", "TEXT"],
                  output_names=["LOGITS_PER_IMAGE", "LOGITS_PER_TEXT"], export_params=True)

/opt/miniconda/envs/yz_openclip/lib/python3.10/site-packages/open_clip/coca_model.py:137: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  text = text[:, :-1] if embed_cls else text # make space for CLS token


RuntimeError: shape '[255, 36, 64]' is invalid for input of size 195840

In [9]:
# caption prediction
image_path = '/mnt/eds_share/share/Spine2D/GlobusSrgMapData_crop_square/test/images/anon_d4730414.dcm_lateral_full_lumbar_thoracic.jpg'
im = Image.open(image_path).convert("RGB")
im = im.resize((224, 224), Image.Resampling.LANCZOS)
im = transform(im).unsqueeze(0)

with torch.no_grad(), torch.cuda.amp.autocast():
    generated = model.generate(im)

print(open_clip.decode(generated[0]).split("<end_of_text>")[0].replace("<start_of_text>", ""))

anteroposterior thoracic lumbar 


In [10]:
# Load standard CLIP model, image, text on cpu
import open_clip
from PIL import Image
import numpy as np
import torch
# onnx cannot work with cuda
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name="coca_ViT-L-14",
    pretrained="/mnt/eds_share/Users/yilu.zhou/Development/log/open_clip_GlobusSrgMapData_crop_square/2023_08_08-22_02_21-model_coca_ViT-L-14-lr_1e-06-b_32-j_4-p_amp/checkpoints/epoch_78.pt"
)
# model, preprocess = clip.load("ViT-B/32", device="cpu", jit=False)

# batch first
image_path = '/mnt/eds_share/share/Spine2D/GlobusSrgMapData_crop_square/test/images/anon_d4730414.dcm_lateral_full_lumbar_thoracic.jpg'
im = Image.open(image_path).convert("RGB")
im = im.resize((224, 224), Image.Resampling.LANCZOS)
image = preprocess(im).unsqueeze(0).cpu() # [1, 3, 224, 224]
# image = preprocess(Image.open("CLIP.png")).unsqueeze(0).cpu() # [1, 3, 224, 224]
image_onnx = image.detach().cpu().numpy().astype(np.float32)

# batch first
tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
sentences = ["thoracic", "lumbar", "thoracic lumbar"]
text = tokenizer(sentences).cpu()
# text = clip.tokenize(["a diagram", "a dog", "a cat"]).cpu() # [3, 77]
text_onnx = text.detach().cpu().numpy().astype(np.int32)

In [13]:
# https://github.com/pytorch/pytorch/issues/97262
import onnxscript

# Assuming you use opset17
from onnxscript.onnx_opset import opset13 as op

custom_opset = onnxscript.values.Opset(domain="torch.onnx", version=1)


@onnxscript.script(custom_opset)
def ScaledDotProductAttention(
    query,
    key,
    value,
    dropout_p,
):
    # Swap the last two axes of key
    key_shape = op.Shape(key)
    key_last_dim = key_shape[-1:]
    key_second_last_dim = key_shape[-2:-1]
    key_first_dims = key_shape[:-2]
    # Contract the dimensions that are not the last two so we can transpose
    # with a static permutation.
    key_squeezed_shape = op.Concat(
        op.Constant(value_ints=[-1]), key_second_last_dim, key_last_dim, axis=0
    )
    key_squeezed = op.Reshape(key, key_squeezed_shape)
    key_squeezed_transposed = op.Transpose(key_squeezed, perm=[0, 2, 1])
    key_transposed_shape = op.Concat(key_first_dims, key_last_dim, key_second_last_dim, axis=0)
    key_transposed = op.Reshape(key_squeezed_transposed, key_transposed_shape)

    embedding_size = op.CastLike(op.Shape(query)[-1], query)
    scale = op.Div(1.0, op.Sqrt(embedding_size))

    # https://github.com/pytorch/pytorch/blob/12da0c70378b5be9135c6fda62a9863bce4a4818/aten/src/ATen/native/transformers/attention.cpp#L653
    # Scale q, k before matmul for stability see https://tinyurl.com/sudb9s96 for math
    query_scaled = op.Mul(query, op.Sqrt(scale))
    key_transposed_scaled = op.Mul(key_transposed, op.Sqrt(scale))
    attn_weight = op.Softmax(
        op.MatMul(query_scaled, key_transposed_scaled),
        axis=-1,
    )
    attn_weight, _ = op.Dropout(attn_weight, dropout_p)
    return op.MatMul(attn_weight, value)


def custom_scaled_dot_product_attention(g, query, key, value, attn_mask, dropout, is_causal, scale=None):
    return g.onnxscript_op(ScaledDotProductAttention, query, key, value, dropout).setType(value.type())


torch.onnx.register_custom_op_symbolic(
    symbolic_name="aten::scaled_dot_product_attention",
    symbolic_fn=custom_scaled_dot_product_attention,
    opset_version=18,
)

In [5]:
import onnxscript, torch
from torch.onnx._internal import _beartype, jit_utils, registration
# There are three opset version needed to be aligned
# This is (1) the opset version in ONNX function
from onnxscript.onnx_opset import opset15 as op
opset_version = 13

x = torch.randn(1, 2, 3, 4, requires_grad=True)
model = torch.nn.SELU()

custom_opset = onnxscript.values.Opset(domain="onnx-script", version=1)

@onnxscript.script(custom_opset)
def Selu(X):
    alpha = 1.67326  # auto wrapped as Constants
    gamma = 1.0507
    alphaX = op.CastLike(alpha, X)
    gammaX = op.CastLike(gamma, X)
    neg = gammaX * (alphaX * op.Exp(X) - alphaX)
    pos = gammaX * X
    zero = op.CastLike(0, X)
    return op.Where(X <= zero, neg, pos)

# setType API provides shape/type to ONNX shape/type inference
def custom_selu(g: jit_utils.GraphContext, X):
    return g.onnxscript_op(Selu, X).setType(X.type())

# Register custom symbolic function
# There are three opset version needed to be aligned
# This is (2) the opset version in registry
torch.onnx.register_custom_op_symbolic(
    symbolic_name="aten::selu",
    symbolic_fn=custom_selu,
    opset_version=opset_version,
)

# There are three opset version needed to be aligned
# This is (2) the opset version in exporter
torch.onnx.export(
    model,
    x,
    "model.onnx",
    opset_version=opset_version,
    # only needed if you want to specify an opset version > 1.
    custom_opsets={"onnx-script": 1}
)

In [14]:
# Create CLIP-ONNX object to convert model to onnx
from clip_onnx import clip_onnx

visual_path = "clip_visual.onnx"
textual_path = "clip_textual.onnx"

onnx_model = clip_onnx(model, visual_path=visual_path, textual_path=textual_path)
onnx_model.convert2onnx(image, text, verbose=True)
# ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
onnx_model.start_sessions(providers=["CPUExecutionProvider"]) # cpu mode

#Use for standard CLIP API. Batch inference
image_features = onnx_model.encode_image(image_onnx)
text_features = onnx_model.encode_text(text_onnx)

logits_per_image, logits_per_text = onnx_model(image_onnx, text_onnx)
probs = logits_per_image.softmax(dim=-1).detach().cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421067 0.00299571]]

# https://github.com/pytorch/pytorch/issues/96944

[CLIP ONNX] Start convert visual model


UnsupportedOperatorError: Exporting the operator 'aten::scaled_dot_product_attention' to ONNX opset version 13 is not supported. Support for this operator was added in version 14, try exporting with this version.